In [1]:
# import necessary packages
import pandas as pd
import os

In [2]:
%load_ext watermark
%watermark

2018-12-13T14:58:47-07:00

CPython 3.6.5
IPython 7.1.1

compiler   : GCC 4.2.1 (Apple Inc. build 5666) (dot 3)
system     : Darwin
release    : 18.2.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [3]:
# import raw data
raw_data = pd.read_csv('../raw_data/student-mat.csv')

In [4]:
pd.set_option('display.max_columns', None)
raw_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [5]:
# engineer the improved variable
period1 = raw_data.G1
period2 = raw_data.G2
print(type(period1), type(period2))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [6]:
improved_list = []

for n in period1 - period2:
    if n >= 0:
        improved_list.append(0)
    else:
        improved_list.append(1)

raw_data['improved'] = improved_list

assert raw_data['improved'].max() == 1 and raw_data['improved'].min() == 0
assert len(raw_data['improved']) == 395

In [7]:
# engineer the pass/fail response variable

pass_fail = []

for n in raw_data.G3:
    if n >= 10:
        pass_fail.append(1)
    else:
        pass_fail.append(0)
        
raw_data['passed'] = pass_fail

assert raw_data['passed'].max() == 1 and raw_data['passed'].min() == 0
assert len(raw_data['passed']) == 395

In [8]:
# Remove missing target variable entries
cleaned_data = raw_data[~(raw_data['G3']==0)]

assert 0 not in cleaned_data['G3'].values
assert len(cleaned_data) == (395 - len(raw_data[raw_data['G3']==0]))

In [9]:
# export cleaned data to directory
cleaned_data.to_csv('cleaned-data.csv')

assert os.path.exists('./cleaned-data.csv')

In [10]:
cleaned_data.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,improved,passed
count,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000
mean,16.655462,2.795518,2.546218,1.431373,2.042017,0.271709,3.955182,3.246499,3.098039,1.495798,2.330532,3.549020,6.316527,11.268908,11.358543,11.523810,0.344538,0.742297
std,1.268262,1.093999,1.084217,0.686075,0.831895,0.671750,0.885721,1.011601,1.090779,0.919886,1.294974,1.402638,8.187623,3.240450,3.147188,3.227797,0.475884,0.437983
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,5.000000,4.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,2.000000,9.000000,9.000000,9.000000,0.000000,0.000000
50%,17.000000,3.000000,3.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000,0.000000,1.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,14.000000,14.000000,14.000000,1.000000,1.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000,1.000000,1.000000
